In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import mglearn

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline

In [60]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)

In [61]:
scaler = MinMaxScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)

In [62]:
svm = SVC()
svm.fit(X_train_scaled, y_train)
X_test_scaled=scaler.transform(X_test)
# svm은 0-1 안으로 fit되엇잔아 그러면
# ** X_test도 일관성 위하여 transform으로 변환되도록 X_test_scaled임을 기억하기

In [63]:
svm.score(X_test_scaled, y_test)

0.9790209790209791

## 최적화

* ~ 서치

In [64]:
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100],
              "gamma" :[0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(SVC(), param_grid=param_grid) #응 파라미터는 저기있는거 싹다돌릴거야, svc돌릴때 bestparam을 찾으라고~ 그리드서치 할건데 svc모델에서 찾아보라궁
grid.fit(X_train_scaled, y_train)
print(grid.best_score_)
grid.score(X_test_scaled, y_test)
grid.best_params_

0.9788508891928865


{'C': 1, 'gamma': 1}

## 최적화

* 파이프라인 구축

> make_pipeline은 pipeline으로 만든 하나짜리들을 ㅇㅇㅇ서치 (ex)그리드서치 같은데 집어넣어서 병렬로 돌려내겠다는 의미

In [65]:
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())]) #순서대로, 1.스케일러 한게 그대로 2.svc에 넘어가게 됨 "" 안은 약간 이름짓는거 라고 생각함됨 또쓸거니까
pipe.fit(X_train, y_train) #주의. *여기는 scaled안된 그냥 train값을 넣는다 !!!! MinMaxScaler를 통과할거기때문에 그냥값 넣음
pipe.score(X_test, y_test)

0.9790209790209791

In [66]:
param_grid = {"svm__C": [0.001, 0.01, 0.1, 1, 10, 100],
              "svm__gamma" :[0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
grid.score(X_test_scaled, y_test)
print(grid.best_score_, grid.best_params_)

0.9741450068399453 {'svm__C': 10, 'svm__gamma': 0.1}


In [67]:
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))
pipe_short

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))])

In [68]:
pipe_short.steps 
#어머 이름을 잡아주노

[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))]

In [69]:
pipe_short=make_pipeline(StandardScaler(), PCA(n_components=2), StandardScaler())
pipe_short.fit(cancer.data)
#이름이 정해진걸 볼 수 있음

Pipeline(steps=[('standardscaler-1', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('standardscaler-2', StandardScaler())])

In [70]:
components=pipe_short.named_steps["pca"].components_
#파이프라인 스텝 중에서 pca일때 속성을 뱉어봐 * components_ * 
print(components)

[[ 0.219  0.104  0.228  0.221  0.143  0.239  0.258  0.261  0.138  0.064
   0.206  0.017  0.211  0.203  0.015  0.17   0.154  0.183  0.042  0.103
   0.228  0.104  0.237  0.225  0.128  0.21   0.229  0.251  0.123  0.132]
 [-0.234 -0.06  -0.215 -0.231  0.186  0.152  0.06  -0.035  0.19   0.367
  -0.106  0.09  -0.089 -0.152  0.204  0.233  0.197  0.13   0.184  0.28
  -0.22  -0.045 -0.2   -0.219  0.172  0.144  0.098 -0.008  0.142  0.275]]


### 전처리와 모델 매개변수를 위한 그리드 서치

In [71]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep=r"\s+", skiprows=22, header=None) #그냥 data_url하니까 위쪽에 줄글땜에 파싱이 안됨 그래서 스킵로우하겠어 헤더는 없더라고..^^명시해주자
#sep=r"\s+" string=문자열 구획으로 나누것어. csv 안에 , 기준으로 원래 구획이 되는데 그게 없어서 안되니까 내가 sep을 만들어줘야지
#sep 매개변수에 r"\s+"를 사용하면, 데이터 파일에서 공백을 구분자로 인식하여 열을 분리함
raw_df

,0,1,2,3,...,7,8,9,10
0,6.32e-03,18.00,2.31,0.0,...,4.09,1.0,296.0,15.3
1,3.97e+02,4.98,24.00,NaN,...,NaN,NaN,NaN,NaN
2,2.73e-02,0.00,7.07,0.0,...,4.97,2.0,242.0,17.8
3,3.97e+02,9.14,21.60,NaN,...,NaN,NaN,NaN,NaN
4,2.73e-02,0.00,7.07,0.0,...,4.97,2.0,242.0,17.8
...,...,...,...,...,...,...,...,...,...
1007,3.97e+02,5.64,23.90,NaN,...,NaN,NaN,NaN,NaN
1008,1.10e-01,0.00,11.93,0.0,...,2.39,1.0,273.0,21.0
1009,3.93e+02,6.48,22.00,NaN,...,NaN,NaN,NaN,NaN
1010,4.74e-02,0.00,11.93,0.0,...,2.50,1.0,273.0,21.0


In [73]:
# 책에 있는 예제라서 해봤지만 가능하면 전처리는 엑셀이나 구글스프레드시트 하기
# vscode에서 텍스트 전처리를 진행함

data = np.hstack([raw_df.values[::2, :],raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [74]:
pipe = make_pipeline(StandardScaler(), PolynomialFeatures(), Ridge())
pipe
#standardScaler - 평균 표준편차 안에 밀어넣는 종모양으로 데이터를 집어넣게 되니까
#특성을 잃어버리고 종모양 근처에 있게끔 설게하게 됨 
#종모양 가까이 있는 애들이 아니고, 끝에 있는. 분산하려고 하는 애들이 더 유의미한 데이터포인트라고 할 수 있음
#회귀 쓸건데 안틀리고싶어 평균으로 넣어가지고 밖에 나와있는 애들 볼 수 있으려면
#회귀에 MinMax 하면 특정 범위 ex(0.1) 안에 다 밀어넣어지니까 튀는 애들이 아예 안나오자나
#회귀때 MinMax가 아니고 Standard를 써야한다는거
#근데 종모양 안에 다 넣어놓으면 분산이 안보일수있기때문에 그런애들을 잡아줄려고 
#polynomial로 다항식으로 튀겨주면 분산있는 애들이 눈에 보일수있음 
#다항식 튀겨줘도 어차피 그 경향성, 일차방정식일때 회귀의 경향성 자체는 안변하기때문에. 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures()),
                ('ridge', Ridge())])

In [77]:
# 매우 중요한부분 
# 전처리에 필요한 매개변수도 여기서 건드려줄수있어서 ex)다항식 차수 조정

param_grid = {"polynomialfeatures__degree":[1,2,3], #다항식 차수 말하는거임 1(본래 선형), 2차, 3차.. 차수가 늘어나도 선형의 경향성은 유지한다
              "ridge__alpha":[0.001, 0.01, 0.1, 1, 10, 100]}

In [82]:
#그리드 서치에 파이프를 집어넣는 것 :
# 파이프라인 내의 모든 단계에 대해 하이퍼파라미터 튜닝을 수행, 모든 단계에서 최적의 파람을 잡아낸다고
grid=GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1) #n_jobs=-1 다쓰라는거야 cpu가용자원 다써라.. 원래는 1만 쓰거든
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('ridge', Ridge())]),
             n_jobs=-1,
             param_grid={'polynomialfeatures__degree': [1, 2, 3],
                         'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]})

In [83]:
grid.best_params_

{'polynomialfeatures__degree': 2, 'ridge__alpha': 10}

In [84]:
grid.score(X_test, y_test)

0.8054402042295686

### 모델 선택을 위한 파이프라인

In [85]:
pipe = Pipeline([("preprocessing", StandardScaler()), ("classifier",  SVC())], memory="cache_folder") #memory="cache_folder" 원 데이터 변경사항 없을때 제일마지막에 쓰기

In [86]:
param_grid = [
     {"classifier":[SVC()], 
      "preprocessing":[StandardScaler()], 
      "classifier__C" : [0.001, 0.01, 0.1, 1, 10, 100]},
     {"classifier":[RandomForestClassifier()], 
      "preprocessing":[None], 
      "classifier__max_features" : [1,2,3]}
]

In [87]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)

In [88]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)



C:\Users\user\AppData\Roaming\Python\Python39\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'preprocessing': [StandardScaler()]},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__max_features': [1, 2, 3],
                          'preprocessing': [None]}])

In [89]:
grid.best_params_

{'classifier': SVC(), 'classifier__C': 1, 'preprocessing': StandardScaler()}

In [90]:
grid.score(X_test, y_test)

0.972027972027972

### 텍스트 데이터 다루기

> 텍스트에서 중요한 거 "오더" : 텍스트의 오더를 기준으로 텍스트를 숫자로 바꿔주는 알고리즘을 알아야 함

> 여기서 필요한건 원핫인코딩이 아니라 embedding

> 한국어는 오더가 안 중요하기 때문에(도치해도 같은 의미) 별도처리
